# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import pandas as pd
import numpy as np

import itertools

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/FakeNewsChallenge"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'/glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
train_bodies = pd.read_csv(project_path+"/train_bodies.csv")
train_stances = pd.read_csv(project_path+"/train_stances.csv")

In [9]:
train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [10]:
train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [11]:
dataset = train_bodies.merge(train_stances, on='Body ID')

print("MERGED TRAIN DATA\n" + '-' * 17)
print("Shape   : {}".format(dataset.shape))
print("Columns : [ {} ]".format(", ".join(dataset.columns)))

MERGED TRAIN DATA
-----------------
Shape   : (49972, 4)
Columns : [ Body ID, articleBody, Headline, Stance ]



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [12]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [17]:
texts = []

texts = dataset['articleBody'].tolist() + dataset['Headline'].tolist()
print(len(texts))

tokenizer.fit_on_texts(texts)

99944


In [18]:
len(tokenizer.word_index)

27873

In [19]:
tokenizer.document_count

99944

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
article_texts = dataset['articleBody'].tolist()

In [0]:
article_sents = []

for article in article_texts:
  article_sents.append(nltk.sent_tokenize(article))

## Check 2:

first element of texts and articles should be as given below. 

In [22]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [23]:
article_sents[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
def trim_or_pad(lst, n):
  return lst[:n] + [0]*(n-len(lst))

In [25]:
article_encodings = []

for i in range(len(article_sents)):
  sentence_encodings = []
  n_sents = 0
  for sentence in article_sents[i]:
    # Limit number of sentences to MAX_SENTS
    if(n_sents < MAX_SENTS):
      print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(tokenizer.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an article has number of sentences less than MAX_SENTS 
  while(n_sents < MAX_SENTS):
    print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  article_encodings.append(sentence_encodings)

In [26]:
article_encodings = np.array(article_encodings)
article_encodings.shape

(49972, 20, 20)

### Check 3:

Accessing first element in data should give something like given below.

In [27]:
article_encodings[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_headline = dataset['Headline'].tolist()

In [0]:
article_headline = []

for headline in texts_headline:
  article_headline.append(nltk.sent_tokenize(headline))

In [30]:
headline_encodings = []

for i in range(len(article_headline)):
  sentence_encodings = []
  n_sents = 0
  for sentence in article_headline[i]:
    # Limit number of sentences to MAX_SENTS_HEADING
    if(n_sents < MAX_SENTS_HEADING):
      print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(tokenizer.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an headline has number of sentences less than MAX_SENTS_HEADING 
  while(n_sents < MAX_SENTS_HEADING):
    print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  headline_encodings.append(sentence_encodings)

In [31]:
headline_encodings = np.array(headline_encodings)
headline_encodings.shape

(49972, 1, 20)

In [32]:
headline_encodings[0, :, :]

array([[  718,   206,   343,  7134,   193,    34,  1338, 11554,    21,
          233,   686,     0,     0,     0,     0,     0,     0,     0,
            0,     0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [33]:
labels = pd.get_dummies(dataset['Stance'])
labels.columns

Index(['agree', 'disagree', 'discuss', 'unrelated'], dtype='object')

In [34]:
labels = labels.values
labels.shape

(49972, 4)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [35]:
print("Shape of articles tensor : {}".format(article_encodings.shape))
print("Shape of headlines tensor : {}".format(headline_encodings.shape))
print("Shape of labels tensor : {}".format(labels.shape))

Shape of articles tensor : (49972, 20, 20)
Shape of headlines tensor : (49972, 1, 20)
Shape of labels tensor : (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(article_encodings.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
article_encodings = article_encodings[indices]

headline_encodings = headline_encodings[indices]

labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
X_train, X_val, y_train, y_val = train_test_split(article_encodings, labels, test_size=0.2, random_state=10)

In [0]:
X_heading_train, X_heading_val, y_heading_train, y_heading_val = train_test_split(headline_encodings, labels, test_size=0.2, random_state=10)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [40]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [41]:
## load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NB_WORDS, 100))


for word, i in tokenizer.word_index.items():
  if i >= MAX_NB_WORDS: 
    continue

  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
#done in the starting cell

### Model

In [43]:
X_train = np.reshape(X_train, (X_train.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_train.shape)

X_val = np.reshape(X_val, (X_val.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_val.shape)

print(y_train.shape)
print(y_val.shape)

(39977, 400)
(9995, 400)
(39977, 4)
(9995, 4)


In [44]:
X_heading_train = np.reshape(X_heading_train, (X_heading_train.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_heading_train.shape)

X_heading_val = np.reshape(X_heading_val, (X_heading_val.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_heading_val.shape)

print(y_heading_train.shape)
print(y_heading_val.shape)

(39977, 20)
(9995, 20)
(39977, 4)
(9995, 4)


In [45]:
input_ = Input(shape=((MAX_SENTS * MAX_SENT_LENGTH)+20, ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0822 03:50:12.124344 140447265236864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 03:50:12.160563 140447265236864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 03:50:12.166759 140447265236864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 03:50:12.177484 140447265236864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0822 03:50:12.178194 1404472652

In [0]:
input_ = Input(shape=((MAX_SENTS * MAX_SENT_LENGTH), ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model1 = Model(inputs=input_, outputs=x)
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 420)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 420, 100)          2000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 420, 200)          160800    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total para

### Compile and fit the model

In [49]:
model1.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_val, y_val))

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 920s 23ms/step - loss: 0.3255 - acc: 0.8694 - val_loss: 0.2905 - val_acc: 0.8821
Epoch 2/5
39977/39977 [==============================] - 906s 23ms/step - loss: 0.2747 - acc: 0.8902 - val_loss: 0.2618 - val_acc: 0.8959
Epoch 3/5
39977/39977 [==============================] - 905s 23ms/step - loss: 0.2525 - acc: 0.9001 - val_loss: 0.2513 - val_acc: 0.9018
Epoch 4/5
39977/39977 [==============================] - 938s 23ms/step - loss: 0.2408 - acc: 0.9059 - val_loss: 0.2441 - val_acc: 0.9036
Epoch 5/5
39977/39977 [==============================] - 945s 24ms/step - loss: 0.2336 - acc: 0.9082 - val_loss: 0.2400 - val_acc: 0.9067


In [0]:
#The above model is training only with article body but the real problem is to comparing heading and body to detect the stance 
#hence concatenating the x_train and x_headline_train

In [0]:
df1 = pd.DataFrame(X_train) 

In [0]:
df2 = pd.DataFrame(X_heading_train)

In [0]:
df3 = pd.DataFrame(X_val)
df4 = pd.DataFrame(X_heading_val)

In [0]:
x_train_con = pd.concat([df1, df2], axis=1)
x_val_con = pd.concat([df3, df4], axis=1)

In [54]:
x_val_con.shape

(9995, 420)

In [55]:
model.fit(x_train_con, y_train, batch_size=64, epochs=5, validation_data=(x_val_con, y_val))

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 995s 25ms/step - loss: 0.3007 - acc: 0.8788 - val_loss: 0.2261 - val_acc: 0.9130
Epoch 2/5
39977/39977 [==============================] - 991s 25ms/step - loss: 0.1834 - acc: 0.9268 - val_loss: 0.1411 - val_acc: 0.9436
Epoch 3/5
39977/39977 [==============================] - 1002s 25ms/step - loss: 0.1300 - acc: 0.9482 - val_loss: 0.1101 - val_acc: 0.9552
Epoch 4/5
39977/39977 [==============================] - 1003s 25ms/step - loss: 0.0999 - acc: 0.9614 - val_loss: 0.0915 - val_acc: 0.9639
Epoch 5/5
39977/39977 [==============================] - 1008s 25ms/step - loss: 0.0805 - acc: 0.9683 - val_loss: 0.0829 - val_acc: 0.9686


## Build the same model with attention layers included for better performance (Optional)

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

        def build(self, input_shape):
          assert len(input_shape) == 3

          self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
          self.features_dim = input_shape[-1]

          if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
          else:
            self.b = None

          self.built = True

        def compute_mask(self, input, input_mask=None):
          return None
        def call(self, x, mask=None):
          features_dim = self.features_dim
          step_dim = self.step_dim

          eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

          if self.bias:
            eij += self.b

          eij = K.tanh(eij)

          a = K.exp(eij)

          if mask is not None:
            a *= K.cast(mask, K.floatx())

          a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

          a = K.expand_dims(a)
          weighted_input = x * a
          return K.sum(weighted_input, axis=1)
        def compute_output_shape(self, input_shape):
          return input_shape[0],  self.features_dim

     

In [0]:
maxlen = 150

In [0]:
input_ = Input(shape=((MAX_SENTS * MAX_SENT_LENGTH)+20, ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = Attention(maxlen)(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 420)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 420, 100)          2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 420, 200)          160800    
_________________________________________________________________
attention_1 (Attention)      (None, 420, 200)          0         
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

## Fit the model and report the accuracy score for the model with attention layer (Optional)

In [60]:
model.fit(x_train_con, y_train, batch_size=64, epochs=5, validation_data=(x_val_con, y_val))

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 999s 25ms/step - loss: 0.2934 - acc: 0.8824 - val_loss: 0.2109 - val_acc: 0.9172
Epoch 2/5
39977/39977 [==============================] - 993s 25ms/step - loss: 0.1782 - acc: 0.9285 - val_loss: 0.1454 - val_acc: 0.9427
Epoch 3/5
39977/39977 [==============================] - 990s 25ms/step - loss: 0.1309 - acc: 0.9475 - val_loss: 0.1194 - val_acc: 0.9519
Epoch 4/5
39977/39977 [==============================] - 994s 25ms/step - loss: 0.0990 - acc: 0.9604 - val_loss: 0.0921 - val_acc: 0.9636
Epoch 5/5
39977/39977 [==============================] - 1000s 25ms/step - loss: 0.0805 - acc: 0.9685 - val_loss: 0.0767 - val_acc: 0.9709
